# `anato-mesh` in Jupyter Notebook 

`anato_mesh.py` contains the main functions for calculating the partition-level curvatures using python to reproduce the algorithm originally published by K. Khabaz here: https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1011815

In [1]:
from anato_mesh import *
from anato_viz import *
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [14]:
#- Pocivavsek lab parent directory
parent_path = r'Z:\aorta\thoracic'
#- Cohort filter
group_str = ['KP', 'KT', 'SA']
file_str = ['SA20_1', 'SA25_1', 'SA41_1', 'SA49_1', 'KP31_3', 'KP16_8', 'KP16_6', 'KT4_1', 'KT9_2', 'KT22_1', 'KT25_1', 'KP4_3', 'KP3_2', 'KT11_1']
#- Mesh filter
#file_str = ['M1.', 'M5.', 'M10.']

$$
\begin{array}{|c|c|}
\hline
\textbf{Quantities} & \textbf{Equation} \\
\hline
\text{Gaussian} & k_1 \cdot k_2 \\
\hline
\text{Mean} & 0.5 \cdot (k_1 + k_2) \\
\hline
\text{IntGaussian} & (k_1 \cdot k_2) \cdot A \\
\hline
\text{IntMeanSquared} & \left(0.5 \cdot (k_1 + k_2)\right)^2 \cdot A \\
\hline
\text{Willmore} & 4 \cdot \left(0.5 \cdot (k_1 + k_2)\right)^2 - 2 \cdot (k_1 \cdot k_2) \\
\hline
\text{IntWillmore} & 4 \cdot \left(0.5 \cdot (k_1 + k_2)\right)^2 \cdot A - 2 \cdot (k_1 \cdot k_2) \cdot A \\
\hline
\text{Casorati} & \sqrt{0.5 \cdot (k_1^2 + k_2^2)} \\
\hline
\text{ShapeIndex} & \frac{2}{\pi} \cdot \arctan\left(\frac{k_2 + k_1}{k_2 - k_1}\right) \\
\hline
\end{array}
$$



You must specify at least one variable in `quantitites` and must also define the `point_removal` method. When in doubt, use the `curvature` method because it is applicable to all geometries. The `thoracic` specific point removal process is still buggy, overly empirical, and is being fixed. If the mesh is ready to be processed as is, then set `None`.

In [15]:
quantities = ['Gaussian', 'IntGaussian', 'Casorati']
point_removal = 'curvature'

The number of surface partitions for each surface is determined by the following equation. `m_set` provides an easily adjustable parameter for further partition scaling and allows you to calculate more than one scaling at once. The optimized value for the thoracic aorta TEVAR dataset is `m=1`.

$$partitions = m \times \left(\frac{SA}{R^2}\right)$$

In [16]:
m_set = [0.5, 1, 5]

#### Executing the batch run: 

In [17]:
results, scan_dict = GetAnatoMeshResults(parent_path, group_str, file_str, point_removal, quantities, m_set)

Organizing paths and file names:
Adding KP16_8_M1.parquet
Adding KP16_8_M10.parquet
Adding KP16_8_M5.parquet
Adding KP3_2_M1.parquet
Adding KP3_2_M10.parquet
Adding KP3_2_M5.parquet
Adding KP31_3_M1.parquet
Adding KP31_3_M10.parquet
Adding KP31_3_M5.parquet
Adding KP4_3_M1.parquet
Adding KP4_3_M10.parquet
Adding KP4_3_M5.parquet
Adding KT11_1_M10.parquet
Adding KT11_1_M5.parquet
Adding KT22_1_M10.parquet
Adding KT22_1_M5.parquet
Adding KT25_1_M10.parquet
Adding KT25_1_M5.parquet
Adding KT4_1_M10.parquet
Adding KT4_1_M5.parquet
Adding KT9_2_M10.parquet
Adding KT9_2_M5.parquet
Adding SA20_1_M10.parquet
Adding SA20_1_M5.parquet
Adding SA25_1_M10.parquet
Adding SA25_1_M5.parquet
Adding SA41_1_M10.parquet
Adding SA41_1_M5.parquet
Adding SA49_1_M10.parquet
Adding SA49_1_M5.parquet
Starting GetAortaMeshResults: the top most progress bar is for all calculations and the progress bars below are for parallel processes.


For KP16_8_M1.parquet, num_patches set to 58 with 103778 elements.
For KP16_8_M1.parquet, num_patches set to 585 with 103778 elements.
For KP16_8_M1.parquet, num_patches set to 117 with 103778 elements.
For KP16_8_M10.parquet, num_patches set to 110 with 10608 elements.
For KP16_8_M10.parquet, num_patches set to 55 with 10608 elements.
For KP16_8_M5.parquet, num_patches set to 113 with 20654 elements.
For KP16_8_M10.parquet, num_patches set to 550 with 10608 elements.
For KP16_8_M5.parquet, num_patches set to 56 with 20654 elements.
For KP16_8_M5.parquet, num_patches set to 565 with 20654 elements.
For KP3_2_M1.parquet, num_patches set to 112 with 139399 elements.
For KP3_2_M1.parquet, num_patches set to 56 with 139399 elements.
For KP3_2_M1.parquet, num_patches set to 560 with 139399 elements.
For KP3_2_M10.parquet, num_patches set to 54 with 12566 elements.
For KP3_2_M10.parquet, num_patches set to 108 with 12566 elements.
For KP3_2_M5.parquet, num_patches set to 55 with 24874 elemen

### If you just want data, stop here. 
#### Everything after this are postprocessing steps including normalization, meta-data integration, visualization, and exporting.

Normalizing the results. This works by first splitting up the compound name using the `SplitUpScanName` function and then normalizing the variables in the 2nd and 3rd function inputs to the `GroupsNormalize` function by the group of datapoints that contain the string sequence in the 4th function input. This is normally `KY` patients for TEVAR and `JX` patients for EVAR in our data. 

In [ ]:
norm_group = 'KY'
xaxis_var = 'Casorati_Mean'
yaxis_var = 'IntGaussian_Fluct'
results_norm = GroupsNormalize(SplitUpScanName(results), xaxis_var, yaxis_var, norm_group)

Merging meta data files into the anato-mesh results.

In [ ]:
#- Meta data integration 
directory = 'Z:\\aorta\\meta_data'
file_name = 'MetaThoracic.xlsx'

#- Cohort meta data group names
cohort_list = ['UC_NORMAL', 'UC_PEDS', 'UC_TEVAR', 'ENDOSPAN', 'MEDTRONIC', 'GORE_801', 'GORE_802', 'GORE_803']

#- Integer columns to be converted to strings for discrete visualization
cat_columns = ['Label', 'Outcome']

In [ ]:
results_meta = MergeMetaData(directory, file_name, cohort_list, results_norm, cat_columns)

Ploting and saving the data. 

In [ ]:
fig = px.scatter(results_meta, x='Mean_Radius', y='IntGaussian_Fluct_Norm', color='Label', hover_data=['ScanName'])
fig.show()

In [ ]:
SaveResultsToXLSX(r'Z:\aorta\datasets', 'AllScaleSampledSpring25.xlsx', results)

In [ ]:
SaveScanDictToXLSX(r'Z:\aorta\...', 'per_patch_data.xlsx', scan_dict) 